In [1]:
commentCountUrl = 'https://comment5.news.sina.com.cn/cmnt/count?format=json&newslist=gn:comos-{}:0'
commentCountUrl2 = 'https://comment.sina.com.cn/page/info?version=1&format=json&channel=cj&newsid=comos-{}'
import re
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
def getCommentCount(newsurl):
    m = re.search('doc-i(.+).shtml',newsurl)  #根据新闻url找出新闻的id
    newsid= m.group(1)
   # print(newsid)
    commentCountUrlFormat = commentCountUrl.format(newsid)   #根据新闻id，组合出评论数量的链接
    commentCountUrlFormat2 = commentCountUrl2.format(newsid)
    #根据评论数量链接，取得评论数
    res = requests.get(commentCountUrlFormat)
    res2 = requests.get(commentCountUrlFormat2)
    res.encoding='utf-8'
   # print(res.text)
    mm = re.search('\"total\":.*?(?=,)',res.text)
    mm2 = re.search('\"total\":.*?(?=,)',res2.text) #这两个路径中只有一个是有返回值的，另一个没有返回值，需要判断
    commentCount1=0
    commentCount2=0
    if mm is not None:
        commentCountStr1 = mm.group(0).lstrip('"total":').strip()
        commentCount1 = int(commentCountStr1)
    if mm2 is not None:
        commentCountStr2 = mm2.group(0).lstrip('"total":').strip()
        commentCount2 = int(commentCountStr2)
    
    #当第二个地址返回正确 ，但是第一个地址返回也不是空，知识内容是0，所以不为空的都要取，然后返回大的评论数
    
    
    if commentCount1 > commentCount2:
        return commentCount1
    else:
        return commentCount2
  

In [2]:
def getAuthor(soup):
    
    show_author = soup.select('.show_author')
    article_editor = soup.select('.article-editor')
    if len(show_author)>0:
        return show_author[0].text.lstrip('责任编辑：') 
    elif len(article_editor)>0:
        return article_editor[0].text.lstrip('责任编辑：')
    else:
        return 'null'

In [3]:
def getNewsDetail(newsurl):
    result = {}
    res = requests.get(newsurl)
    res.encoding='utf-8'
    soup = BeautifulSoup(res.text,'html.parser')
    result['title'] = soup.select('.main-title')[0].text
    result['dt'] = datetime.strptime(soup.select('.date')[0].text,'%Y年%m月%d日 %H:%M')
    result['newssource'] = soup.select('.source')[0].text
    result['article'] = ' '.join([p.text.strip() for p in soup.select('.article p')[:-1]])
    result['editor'] = getAuthor(soup)
    result['commentsCount'] = getCommentCount(newsurl)
    return result

In [4]:
def parseListLinks(pageUrl):
    newsdetails = []
    res = requests.get(pageUrl)
    res.encoding='utf-8'
    jd = json.loads(res.text)
    for ent in jd['result']['data']:
        newsdetails.append(getNewsDetail(ent['url']))
    return newsdetails

In [5]:
def getNewsTotal(start, end):
    pageCommonUrl = 'https://feed.mix.sina.com.cn/api/roll/get?pageid=153&lid=2509&k=&num=10&page={}'
    if str(start).isdigit() and str(end).isdigit():
        if int(start)< int(end):
            news_total = []
            for i in range(int(start),int(end)):
                newsurl = pageCommonUrl.format(i)
                newsary = parseListLinks(newsurl)
                news_total.extend(newsary)
            return news_total
        else:
            return None
    else:
        return None
    

In [6]:
import pandas
df = pandas.DataFrame(getNewsTotal(11,13))
df.head(20)

,article,commentsCount,dt,editor,newssource,title
0,中国平安千亿回购还搞不搞？这可能是现在及未来一段时间，股民看到平安二字后想到的首要问题。 中...,0,2018-12-14 22:29:00,张海营,中国证券报,平安回购方案通过但仍留悬念 马明哲开始布局接班人?
1,原标题：超2亿元！湖南广播电视台接手徐悲鸿画作《愚公移山》 涉足艺术品投资的电广传媒，打算将...,0,2018-12-14 22:29:00,张玉,澎湃新闻,超2亿 湖南广播电视台购徐悲鸿画作《愚公移山》
2,超2亿元！湖南广播电视台接手徐悲鸿画作《愚公移山》 澎湃新闻记者 陈宇曦 综合报道 来源：澎...,0,2018-12-14 22:29:00,张国帅,澎湃新闻,超2亿元!湖南广播电视台接手徐悲鸿画作《愚公移山》
3,首开集团再挂两宗资产转让 合计金额1.05亿元 莫莫/文 据北交所日前披露信息显示，北京首都...,0,2018-12-14 22:28:00,刘万里 SF014,中国网,首开集团再挂两宗资产转让 合计金额1.05亿元
4,新浪科技讯 北京时间12月14日晚间消息，华尔街投行Baird日前发布投资研究报告，将特斯拉...,14,2018-12-14 22:28:00,张宁,新浪科技,投行看好特斯拉盈利前景 纷纷调高目标股价
5,相关新闻：特斯拉员工爆料:马斯克火气大爱炒鱿鱼 员工不敢接近 新浪科技讯 北京时间12月14...,0,2018-12-14 22:27:00,null,新浪科技,特斯拉回应媒体报道：马斯克不会无缘无故解雇员工
6,原标题：四川筠连县女教师涉嫌遭村文书猥亵已立刑案，嫌疑人被刑拘 为四川宜宾市筠连县孔雀乡代课...,0,2018-12-14 22:27:00,闫宏亮,澎湃新闻,四川女教师涉嫌遭村文书猥亵已立案 嫌疑人被刑拘
7,原标题：河南汝州回应“高中强卖校服”：高三学生已获退款 新京报讯（记者 刘名洋 雷燕超）12...,0,2018-12-14 22:26:00,闫宏亮,新京报,河南汝州回应“高中强卖校服”:高三学生已获退款
8,原标题：外媒：欧盟领导人不确定是否还能相信特雷莎·梅 [环球网报道 记者 左甜]彭博社12月...,0,2018-12-14 22:25:00,张玉,环球网,外媒：欧盟领导人不确定是否还能相信特雷莎-梅
9,味千中国CFO刘家豪涉嫌挪用179.5万港元被免职 新京报快讯（记者 张晓荣）12月14日晚...,0,2018-12-14 22:24:00,刘万里 SF014,新京报,味千中国CFO刘家豪涉嫌挪用179.5万港元被免职


In [7]:
df.to_excel('news_result.xlsx')